<a href="https://colab.research.google.com/github/Fernando-Fernandez/ApexTriggers/blob/master/Interacting_with_CodeGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interacting with [CodeGen](https://github.com/salesforce/CodeGen/)

- Originally from afiaka87
- Orginized Colab by mega b#6696
- Modified by Penguin-jpg

In [ ]:
#@markdown # Show gpu information
from IPython.display import clear_output
from google.colab.output import eval_js
eval_js('google.colab.output.setIframeHeight("500")')

!nvidia-smi

Fri Oct  7 16:18:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@markdown # Install requirements
!git clone https://github.com/salesforce/CodeGen
%cd CodeGen
!pip install --upgrade pip setuptools
!pip install -r requirements.txt

Cloning into 'CodeGen'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 166 (delta 74), reused 121 (delta 44), pack-reused 2
Receiving objects: 100% (166/166), 1.35 MiB | 5.27 MiB/s, done.
Resolving deltas: 100% (74/74), done.
/content/CodeGen
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 27.4 MB/s 
     |████████████████████████████████| 1.2 MB 53.1 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 GB 149.5 MB/s eta 0:00:01tcmalloc: large alloc 2041348096 bytes == 0x4032000 @  0x7f80306621e7 0x4b2150 0x4b21dc 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x517947 0x5b41c5 0x58f49e 0x517947 0x5b41c5 0x58f49e 0x517947 0x5b41c5 0x58f49e 0x517947 0x5b41c5 0x58f49e 0x517947 0x58f2a7 0x517947 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x517947 0x5b41c5
tcmalloc: large alloc 2551685120 bytes == 0x7dafa000 @  0x7f8030663615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x517947 0x5b41c5 0x58f49e 0x517947 0x5b41c5 0x58f49e 0x517947 0x5b41c5 0x58f49e 0x517947 0x5b41c5 0x58f49e 0x517947 0x58f2a7 0x517947 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x517947 0x5b41c5 0x58f49e 0x517947
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 GB 135.1 MB/s eta 

In [ ]:
#@markdown # Load model and tokenizer
chosen_model = "codegen-350M-mono" #@param ["codegen-350M-nl", "codegen-350M-multi", "codegen-350M-mono", "codegen-2B-nl", "codegen-2B-multi", "codegen-2B-mono", "codegen-6B-nl", "codegen-6B-multi", "codegen-6B-mono", "codegen-16B-nl", "codegen-16B-multi", "codegen-16B-mono"]
fp16 = True #@param {type:"boolean"}

import os

if not os.path.exists(f'./checkpoints/{chosen_model}'):
  !wget -P checkpoints https://storage.googleapis.com/sfr-codegen-research/checkpoints/{chosen_model}.tar.gz && tar -xvf checkpoints/{chosen_model}.tar.gz -C checkpoints/


import torch
from jaxformer.hf.sample import truncate as do_truncate
from jaxformer.hf.sample import set_env, set_seed, print_time, create_model, create_custom_gpt2_tokenizer, create_tokenizer, sample

# (0) constants

models_nl = ['codegen-350M-nl', 'codegen-2B-nl', 'codegen-6B-nl', 'codegen-16B-nl']
models_pl = ['codegen-350M-multi', 'codegen-2B-multi', 'codegen-6B-multi', 'codegen-16B-multi', 'codegen-350M-mono', 'codegen-2B-mono', 'codegen-6B-mono', 'codegen-16B-mono']
models = models_nl + models_pl


# (2) preamble

set_env()

pad = 50256
device = torch.device('cuda:0')
ckpt = f'./checkpoints/{chosen_model}'

if device.type == "cpu":
  print()
  print("force full precision for cpu!!")
  print()
  fp16 = False


# (3) load

with print_time('loading parameters'):
  model = create_model(ckpt=ckpt, fp16=fp16).to(device)


with print_time('loading tokenizer'):
  if chosen_model in models_pl:
    tokenizer = create_custom_gpt2_tokenizer()
  else:
    tokenizer = create_tokenizer()
  tokenizer.padding_side = 'left'
  tokenizer.pad_token = pad

--2022-10-07 16:25:42--  https://storage.googleapis.com/sfr-codegen-research/checkpoints/codegen-350M-mono.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 142.251.10.128, 74.125.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 656148604 (626M) [application/x-tar]
Saving to: ‘checkpoints/codegen-350M-mono.tar.gz’

codegen-350M-mono.t 100%[===================>] 625.75M  3.79MB/s    in 3m 20s  

2022-10-07 16:29:02 (3.13 MB/s) - ‘checkpoints/codegen-350M-mono.tar.gz’ saved [656148604/656148604]

codegen-350M-mono/
codegen-350M-mono/config.json
codegen-350M-mono/pytorch_model.bin
loading parameters
loading parameters took 23.19s
loading tokenizer


Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

loading tokenizer took 15.69s


In [ ]:
#@markdown # Try out the model
rng_seed = 42 #@param {type:"integer"}
rng_deterministic = True #@param {type:"boolean"}
p = 0.95 #@param {type:"number"}
t = 0.2 #@param {type:"number"}
max_length = 128 #@param {type:"integer"}
batch_size = 1 #@param {type:"integer"}
context = "def calculate_Pi( number_of_digits ):" #@param {type:"string"}

set_seed(rng_seed, deterministic=rng_deterministic)

# (4) sample

with print_time('sampling'):
  completion = sample(device=device, model=model, tokenizer=tokenizer, context=context, pad_token_id=pad, num_return_sequences=batch_size, temp=t, top_p=p, max_length_sample=max_length)[0]
  truncation = do_truncate(completion)

  print('=' * 100)
  print(completion)
  print('=' * 100)
  print(context+truncation)
  print('=' * 100)
    

# !python -m jaxformer.hf.sample --model $chosen_model \
#                  --rng-seed $rng_seed \
#                  --p $p \
#                  --t $t \
#                  --max-length $max_length \
#                  --batch-size $batch_size \
#                  --context '$context'
    

sampling

    Pi = 1
    for i in range(1, number_of_digits+1):
        Pi = Pi * (i)
    return Pi

def calculate_Pi_2( number_of_digits ):
    Pi = 1
    for i in range(1, number_of_digits+1):
        Pi = Pi * (i)
    return Pi

def calculate_Pi_3( number_of_digits ):
    Pi = 1
    for i in range(1, number_of_digits+1):

def calculate_Pi( number_of_digits ):
    Pi = 1
    for i in range(1, number_of_digits+1):
        Pi = Pi * (i)
    return Pi

def calculate_Pi_2( number_of_digits ):
    Pi = 1
    for i in range(1, number_of_digits+1):
        Pi = Pi * (i)
    return Pi


sampling took 15.10s
